## Implementing TF-IDF
<br>
TF-IDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

## Step 0: Getting some data
Let's start by setting up three documents:

In [5]:
import numpy as np 


documents = []

documents.append("""Python is a 2000 made-for-TV horror movie directed by Richard Clabaugh. 
The film features several cult favorite actors, including William
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy,
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean
Whalen. The film concerns a genetically engineered snake, a python, that
escapes and unleashes itself on a small town. It includes the classic final
girl scenario evident in films like Friday the 13th. It was filmed in Los Angeles,
 California and Malibu, California. Python was followed by two sequels: Python
 II (2002) and Boa vs. Python (2004), both also made-for-TV films.""")

documents.append("""Python, from the Greek word (πύθων/πύθωνας), is a genus of
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are
recognised.[2] A member of this genus, P. reticulatus, is among the longest
snakes known.""")

documents.append("""The Colt Python is a .357 Magnum caliber revolver formerly
manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
It is sometimes referred to as a "Combat Magnum".[1] It was first introduced
in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
Colt Python targeted the premium revolver market segment. Some firearm
collectors and writers such as Jeff Cooper, Ian V. Hogg, Chuck Hawks, Leroy
Thompson, Renee Smeets and Martin Dougherty have described the Python as the
finest production revolver ever made.""")

## Step 1 : Implementing a tokenizer

Before counting words, we want to lemmatize everything. This step helps to reduce dimensionnality of our data. 
<br>
Here's simple tokenizer/lemmatizer using NLTK :

In [8]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
stopwords = [l.rstrip() for l in open('stopwords.txt').read()]

def tokenize(document):
    # Put everything to lowercase
    document = document.lower()
    
    # Split document into words (=tokens) 
    tokens = word_tokenize(document)
    
    # Remove all words that are too small
    tokens = [t for t in tokens if len(t) > 2]
    
    # Remove all stopwords
    tokens = [t for t in tokens if t not in stopwords]
    
    # Turn tokens into their base form
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    # Removes all digits
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] 
    
    return tokens


# Tokenize all documents
all_tokens = [tokenize(d) for d in documents]

## Step 2 : Computing TF (Term-frequency)

Here, we're going to compute the term-frequency matrix which is defined as follows: $tf_{i,j} = \frac{n_{i,j}}{\sum_{k} n_{k,j}}$
<br>
Where i is the index of ith word, j is the index of the current document, and $\sum_{k} n_{k,j}$ is the total of words in that document.

First, we want to give every token an unique index.

In [9]:
import itertools

# Get all tokens in a single array and removes duplicates 
vocabulary = list(set(itertools.chain.from_iterable(all_tokens)))

# Dict that maps word: index
word_index_map = {w: i for i, w in enumerate(vocabulary)}

print(word_index_map)

{'described': 0, 'now': 1, 'martin': 2, 'wheaton': 3, 'role': 4, 'classic': 5, 'van': 6, 'directed': 7, 'malibu': 8, 'girl': 9, 'coogan': 10, 'ever': 11, 'have': 12, 'engineered': 13, 'combat': 14, 'snake': 15, 'jenny': 16, 'genus': 17, 'ian': 18, 'movie': 19, 'keith': 20, 'several': 21, 'leroy': 22, 'wesson': 23, 'word': 24, 'colt': 25, 'smeets': 26, 'krueger': 27, 'casper': 28, 'targeted': 29, 'also': 30, 'bowe': 31, 'his': 32, 'two': 33, 'python': 34, 'elm': 35, 'escape': 36, 'nightmare': 37, 'asia': 38, 'town': 39, 'jeff': 40, 'sean': 41, 'made-for-tv': 42, 'smith': 43, 'nonvenomous': 44, 'found': 45, 'fame': 46, 'renee': 47, 'street': 48, 'feature': 49, 'specie': 50, 'boa': 51, 'reticulatus': 52, 'hartford': 53, 'mccarthy': 54, 'same': 55, 'william': 56, 'made': 57, 'πύθων/πύθωνας': 58, 'sometimes': 59, 'for': 60, 'wil': 61, 'whalen': 62, 'itself': 63, 'connecticut': 64, 'manufactured': 65, 'known': 66, 'los': 67, 'sequel': 68, 'collector': 69, 'revolver': 70, 'year': 71, 'best': 

<br>
Now let's cmpute TF for every documents:
<br>

In [15]:
D = len(all_tokens)
W = len(vocabulary)

tf = np.zeros((D, W))
for i in range(D):
    total_words = len(all_tokens[i])
    for t in all_tokens[i]:
        tf[i,word_index_map[t]] += 1/total_words

## Step 3: Computing IDF ( Inverse Document Frequency)

Contrary to the TF frequency matrix, all document are used to compute the IDF. The IDF is computed as follows: $idf_{j} = log(\frac{nb_{document}}{nb_{presence}}) $ 
<br>
where j is the index of the current word $w_j$, $nb_{document}$ is number of document that we are dealing with (3, in our case) and $nb_{presence}$ is the number of document where $w_j$ appears.

In [22]:
import math

idf = np.zeros(W)
for token in vocabulary:
    for tokens in all_tokens:
        if token in tokens:
            idf[word_index_map[token]] += 1

idf = [math.log(D/i) for i in idf]

## Step 4: Computing TF-IDF

The TF-IDF of a word $w_j$ from a particular document $i$ is computed by multiplying its $tf_{i,j}$ with $idf_j$.

In [23]:
tf_idf = tf * idf

## Step 5: Finding the top words of a document

For each document we would like to see what are the top words:

In [31]:
import operator 


for i in range(D):
    # Creates a map between words and their TF-IDF scores (words: tfidf)
    scores = {w: tf_idf[i, word_index_map[w]] for w in vocabulary}
    scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)
    
    print("Top words for document {}".format(i+1))
    for score in scores[:3]:
        print("\tWord: {}, TF-IDF: {}".format(score[0], round(score[1], 5)))
    

Top words for document 1
	Word: film, TF-IDF: 0.05549
	Word: made-for-tv, TF-IDF: 0.02219
	Word: california, TF-IDF: 0.02219
Top words for document 2
	Word: genus, TF-IDF: 0.08451
	Word: word, TF-IDF: 0.04225
	Word: asia, TF-IDF: 0.04225
Top words for document 3
	Word: colt, TF-IDF: 0.04919
	Word: revolver, TF-IDF: 0.04919
	Word: magnum, TF-IDF: 0.04919
